# Balanceamento Estrutural

Importando bibliotecas:

In [ ]:
import networkx as nx
import socnet as sn

from random import choice, random
from itertools import combinations

Configurando bibliotecas:

In [ ]:
sn.node_size = 5
sn.node_color = (255, 255, 255)

sn.edge_width = 1

Definindo constantes:

In [ ]:
NUM_NODES = 15

POSITIVE_COLOR = (0, 0, 255)
NEGATIVE_COLOR = (255, 0, 0)

Inicialização da simulação:

In [ ]:
def set_edge_color(g, n, m):
    if g.edges[n, m]['signal'] == 1:
        g.edges[n, m]['color'] = POSITIVE_COLOR
    else:
        g.edges[n, m]['color'] = NEGATIVE_COLOR


def build_graph(frac_positives=0.5):
    g = nx.complete_graph(NUM_NODES)

    sn.reset_node_colors(g)

    for n, m in g.edges:
        if random() < frac_positives:
            g.edges[n, m]['signal'] = 1
        else:
            g.edges[n, m]['signal'] = -1

        set_edge_color(g, n, m)

    sn.reset_positions(g)

    return g

Passo da simulação:

In [ ]:
def invert(g, n, m):
    g.edges[n, m]['signal'] *= -1

    set_edge_color(g, n, m)


def update(g, mec2_weight=1, mec3_weight=1, mec5_weight=1):
    # Inicializa as pressões.

    for n, m in g.edges():
        g.edges[n, m]['pressure'] = 0

    # Para cada tríade.

    for t in combinations(g.nodes, 3):
        # Lista todas as arestas da tríade.
        edges = [(t[0], t[1]), (t[0], t[2]), (t[1], t[2])]

        # Conta quantas arestas são positivas.
        num_positives = sum(g.edges[n, m]['signal'] == 1 for n, m in edges)

        # Duas positivas significa tríade instável.
        if num_positives == 2:
            for n, m in edges:
                if g.edges[n, m]['signal'] == -1:
                    g.edges[n, m]['pressure'] += mec2_weight # Mecanismo 2
                else:
                    g.edges[n, m]['pressure'] += mec3_weight # Mecanismo 3

        # Zero positivas também significa tríade instável.
        elif num_positives == 0:
            for n, m in edges:
                g.edges[n, m]['pressure'] += mec5_weight # Mecanismo 5

    # Obtém a maior pressão.
    pressure = max(g.edges[n, m]['pressure'] for n, m in g.edges)

    # Se for positiva, inverte uma aresta que esteja
    # sob essa pressão, escolhida aleatoriamente, e
    # devolve True para indicar que houve mudança.
    if pressure > 0:
        n, m = choice([(n, m) for n, m in g.edges if g.edges[n, m]['pressure'] == pressure])
        invert(g, n, m)
        return True

    # Se for zero, devolvendo False para indicar que
    # não houve mudança, ou seja, a rede estabilizou.
    return False

Animação da simulação:

In [ ]:
g = build_graph()

sn.reset_positions(g)

frames = []

while update(g):
    sn.update_positions(g, weight='signal')

    frames.append(sn.generate_frame(g))

print('simulação encerrada, gerando animação...')

sn.show_animation(frames)

Avaliação da simulação:

In [ ]:
def number_components(g):
    h = g.copy()

    for n, m in g.edges:
        if g.edges[n, m]['signal'] == -1:
            h.remove_edge(n, m)

    return nx.number_connected_components(h)


print(number_components(g))